In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML,clear_output
import joblib

In [2]:
splitted_productUserdf = joblib.load('../input/eda-featureextraction-section1/ProductUserdf.pkl')
splitted_productUserdf.rename(columns={'LenOfList':'LengthBeforeSplitting'},inplace=True)
with pd.option_context('display.max_rows', 100, 'display.max_columns', 10,'max_colwidth', None):
    display(HTML(splitted_productUserdf.head().to_html()))

,user_id,product_id,LengthBeforeSplitting,dummyPrediction_goneDay,dummyPrediction_datetime,splitted_allDiffDay,splitted_allGoneDay
0,577243,724112,5,206.000000,2019-07-26,"[[nan, nan, nan, 12.0, 28.0, 55.0, 13.0]]","[[nan, nan, 71.0, 83.0, 111.0, 166.0, 179.0]]"
1,577243,875550,4,215.333333,2019-08-04,"[[nan, nan, nan, nan, 25.0, 59.0, 64.0]]","[[nan, nan, nan, 18.0, 43.0, 102.0, 166.0]]"
2,577243,895941,11,378.800000,2020-01-14,"[[3.0, 71.0, 52.0, 5.0, 8.0, 55.0, 112.0], [nan, nan, nan, nan, 7.0, 5.0, 10.0]]","[[43.0, 114.0, 166.0, 171.0, 179.0, 234.0, 346.0], [nan, nan, nan, 18.0, 25.0, 30.0, 40.0]]"
3,577243,895955,4,390.666667,2020-01-26,"[[nan, nan, nan, nan, 99.0, 158.0, 12.0]]","[[nan, nan, nan, 32.0, 131.0, 289.0, 301.0]]"
4,577243,896036,12,307.454545,2019-11-04,"[[18.0, 10.0, 31.0, 29.0, 4.0, 122.0, 25.0], [nan, nan, nan, 10.0, 6.0, 7.0, 18.0]]","[[61.0, 71.0, 102.0, 131.0, 135.0, 257.0, 282.0], [nan, nan, 2.0, 12.0, 18.0, 25.0, 43.0]]"


In [3]:
#testing how to extract features from "splitted_allDiffDay" and "splitted_allGoneDay"
x =splitted_productUserdf.iloc[2].to_frame().T
leftPart = x.iloc[:,:5]
# rightPart -> "splitted_allDiffDay" and "splitted_allGoneDay"
rightPart = x.iloc[:,5:]
with pd.option_context('display.max_rows', 100, 'display.max_columns', 10,'max_colwidth', None):
    print('leftPart:')
    display(leftPart)
    print('------------------------------------------------')
    print('rightPart:')
    display(rightPart)
print('------------------------------------------------')
print('features')
L=[]
for idx,col in rightPart.iterrows():
    for lst in col:
        L.append(pd.DataFrame(lst)) 
display(L[0])
display(L[1])
print('------------------------------------------------')
print('merging and Concatenating')
merg=leftPart.merge(L[0],how='cross')
display(merg)
print('------------------------------------------------')
display(pd.concat([merg,L[1]],axis=1))

leftPart:


,user_id,product_id,LengthBeforeSplitting,dummyPrediction_goneDay,dummyPrediction_datetime
2,577243,895941,11,378.8,2020-01-14


------------------------------------------------
rightPart:


,splitted_allDiffDay,splitted_allGoneDay
2,"[[3.0, 71.0, 52.0, 5.0, 8.0, 55.0, 112.0], [nan, nan, nan, nan, 7.0, 5.0, 10.0]]","[[43.0, 114.0, 166.0, 171.0, 179.0, 234.0, 346.0], [nan, nan, nan, 18.0, 25.0, 30.0, 40.0]]"


------------------------------------------------
features


...

,0,1,2,3,4,5,6
0,43.0,114.0,166.0,171.0,179.0,234.0,346.0
1,NaN,NaN,NaN,18.0,25.0,30.0,40.0


------------------------------------------------
merging and Concatenating


,user_id,product_id,LengthBeforeSplitting,dummyPrediction_goneDay,dummyPrediction_datetime,0,1,2,3,4,5,6
0,577243,895941,11,378.8,2020-01-14,3.0,71.0,52.0,5.0,8.0,55.0,112.0
1,577243,895941,11,378.8,2020-01-14,NaN,NaN,NaN,NaN,7.0,5.0,10.0


------------------------------------------------


,user_id,product_id,LengthBeforeSplitting,dummyPrediction_goneDay,dummyPrediction_datetime,0,1,2,3,4,5,6,0,1,2,3,4,5,6
0,577243,895941,11,378.8,2020-01-14,3.0,71.0,52.0,5.0,8.0,55.0,112.0,43.0,114.0,166.0,171.0,179.0,234.0,346.0
1,577243,895941,11,378.8,2020-01-14,NaN,NaN,NaN,NaN,7.0,5.0,10.0,NaN,NaN,NaN,18.0,25.0,30.0,40.0


In [4]:
from numba import jit

In [5]:

def make_windowFreatures(dataFrame):
    df = pd.DataFrame()
    for idx,row in dataFrame.iterrows():
        leftPart =  pd.DataFrame(row.iloc[:5]).T
        # rightPart -> "splitted_allDiffDay" and "splitted_allGoneDay"
        rightPart = row.iloc[5:]
        L=[]
        for item in rightPart:
            L.append(pd.DataFrame(item)) 

        merg=leftPart.merge(L[0],how='cross')
        df = pd.concat([df, pd.concat([merg,L[1]],axis=1) ],axis=0)
    return df

In [6]:
%%time
#test extracting features
df = splitted_productUserdf.iloc[1:5].copy()
with pd.option_context('display.max_rows', 100, 'display.max_columns', 20,'max_colwidth', None):
    display(df)
print('result: ')
df = make_windowFreatures(df)
with pd.option_context('display.max_rows', 100, 'display.max_columns', 20,'max_colwidth', None):
    display(df)

,user_id,product_id,LengthBeforeSplitting,dummyPrediction_goneDay,dummyPrediction_datetime,splitted_allDiffDay,splitted_allGoneDay
1,577243,875550,4,215.333333,2019-08-04,"[[nan, nan, nan, nan, 25.0, 59.0, 64.0]]","[[nan, nan, nan, 18.0, 43.0, 102.0, 166.0]]"
2,577243,895941,11,378.800000,2020-01-14,"[[3.0, 71.0, 52.0, 5.0, 8.0, 55.0, 112.0], [nan, nan, nan, nan, 7.0, 5.0, 10.0]]","[[43.0, 114.0, 166.0, 171.0, 179.0, 234.0, 346.0], [nan, nan, nan, 18.0, 25.0, 30.0, 40.0]]"
3,577243,895955,4,390.666667,2020-01-26,"[[nan, nan, nan, nan, 99.0, 158.0, 12.0]]","[[nan, nan, nan, 32.0, 131.0, 289.0, 301.0]]"
4,577243,896036,12,307.454545,2019-11-04,"[[18.0, 10.0, 31.0, 29.0, 4.0, 122.0, 25.0], [nan, nan, nan, 10.0, 6.0, 7.0, 18.0]]","[[61.0, 71.0, 102.0, 131.0, 135.0, 257.0, 282.0], [nan, nan, 2.0, 12.0, 18.0, 25.0, 43.0]]"


result: 


,user_id,product_id,LengthBeforeSplitting,dummyPrediction_goneDay,dummyPrediction_datetime,0,1,2,3,4,5,6,0,1,2,3,4,5,6
0,577243,875550,4,215.333333,2019-08-04,NaN,NaN,NaN,NaN,25.0,59.0,64.0,NaN,NaN,NaN,18.0,43.0,102.0,166.0
0,577243,895941,11,378.8,2020-01-14,3.0,71.0,52.0,5.0,8.0,55.0,112.0,43.0,114.0,166.0,171.0,179.0,234.0,346.0
1,577243,895941,11,378.8,2020-01-14,NaN,NaN,NaN,NaN,7.0,5.0,10.0,NaN,NaN,NaN,18.0,25.0,30.0,40.0
0,577243,895955,4,390.666667,2020-01-26,NaN,NaN,NaN,NaN,99.0,158.0,12.0,NaN,NaN,NaN,32.0,131.0,289.0,301.0
0,577243,896036,12,307.454545,2019-11-04,18.0,10.0,31.0,29.0,4.0,122.0,25.0,61.0,71.0,102.0,131.0,135.0,257.0,282.0
1,577243,896036,12,307.454545,2019-11-04,NaN,NaN,NaN,10.0,6.0,7.0,18.0,NaN,NaN,2.0,12.0,18.0,25.0,43.0


CPU times: user 75 ms, sys: 1.8 ms, total: 76.8 ms
Wall time: 75.9 ms


Applying "make_windowFreatures" function to the all dataset.

In [7]:
wF_extracted_productUserdf = make_windowFreatures(splitted_productUserdf)
with pd.option_context('display.max_rows', 100, 'display.max_columns', 20,'max_colwidth', None):
    display(wF_extracted_productUserdf.head())

,user_id,product_id,LengthBeforeSplitting,dummyPrediction_goneDay,dummyPrediction_datetime,0,1,2,3,4,5,6,0,1,2,3,4,5,6
0,577243,724112,5,206.0,2019-07-26,NaN,NaN,NaN,12.0,28.0,55.0,13.0,NaN,NaN,71.0,83.0,111.0,166.0,179.0
0,577243,875550,4,215.333333,2019-08-04,NaN,NaN,NaN,NaN,25.0,59.0,64.0,NaN,NaN,NaN,18.0,43.0,102.0,166.0
0,577243,895941,11,378.8,2020-01-14,3.0,71.0,52.0,5.0,8.0,55.0,112.0,43.0,114.0,166.0,171.0,179.0,234.0,346.0
1,577243,895941,11,378.8,2020-01-14,NaN,NaN,NaN,NaN,7.0,5.0,10.0,NaN,NaN,NaN,18.0,25.0,30.0,40.0
0,577243,895955,4,390.666667,2020-01-26,NaN,NaN,NaN,NaN,99.0,158.0,12.0,NaN,NaN,NaN,32.0,131.0,289.0,301.0


In [8]:
joblib.dump(wF_extracted_productUserdf,'wF_extracted_productUserdf.pkl')

['wF_extracted_productUserdf.pkl']

In [9]:
wF_extracted_productUserdf.shape

(425560, 19)

I can't continue this notebook right now. There are some mistakes in the type of feature extraction that should be corrected and the window method is probably not suitable, but because I don't have enough time to continue, I will finish this project here.
In addition, the code written here can be a good source for learning.
